# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collection_names = db.list_collection_names()
print(collection_names)

['establishments']


In [6]:
# review the collections in our new database
collection_names = db.list_collection_names()
for collection_name in collection_names:
    print(collection_name)

establishments


In [7]:
# review a document in the establishments collection
establishments_collection = db['establishments']

document = establishments_collection.find_one()
print(document)

{'_id': ObjectId('65da9c796e15b6fd76d7f612'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.08384, 'latitude': 51.490142}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_info = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",  
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
insert_result = establishments_collection.insert_one(new_restaurant_info)


In [11]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("The new restaurant was inserted successfully.")
    print("Inserted document ID:", insert_result.inserted_id)
else:
    print("Failed to insert the new restaurant.")

The new restaurant was inserted successfully.
Inserted document ID: 65dab874816bc24a6a27e54f


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = establishments_collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

print("BusinessTypeID for Restaurant/Cafe/Canteen:", business_type_query)


BusinessTypeID for Restaurant/Cafe/Canteen: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [30]:
# Update the new restaurant with the correct BusinessTypeID
business_type_query = establishments_collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "_id": 0}
)

business_type_id = business_type_query.get("BusinessTypeID")
restaurant_id = "65daa0dd6e15b6fd76d7f617"

In [31]:
# Print the retrieved BusinessTypeID
print("Retrieved BusinessTypeID:", business_type_id)


Retrieved BusinessTypeID: 


In [32]:
# Confirm that the new restaurant was updated
update_result = establishments_collection.update_one(
    {"_id": restaurant_id},
    {"$set": {"BusinessTypeID": business_type_id}}
)

if update_result.modified_count > 0:
    print("The new restaurant was updated successfully.")
else:
    print("The new restaurant was not updated.")


The new restaurant was not updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [43]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})

print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [44]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many({"LocalAuthorityName": "Dover"})

print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 0


In [45]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})

if remaining_dover_count > 0:
    print("There are remaining documents with LocalAuthorityName as 'Dover'.")
else:
    print("There are no remaining documents with LocalAuthorityName as 'Dover'.")


There are no remaining documents with LocalAuthorityName as 'Dover'.


In [46]:
# Check that other documents remain with 'find_one'
remaining_document = establishments_collection.find_one()

if remaining_document:
    print("At least one document remains in the collection:")
    pprint(remaining_document)
else:
    print("No documents remain in the collection.")


At least one document remains in the collection:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65da9c796e15b6fd76d7f612'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude and latitude
batch_size = 1000
total_documents = establishments_collection.count_documents({"geocode": {"$exists": True}})
updated_count = 0
remaining_documents = total_documents

while remaining_documents > 0:
    documents_to_update = establishments_collection.find({"geocode": {"$exists": True}}).limit(batch_size)
    
    for document in documents_to_update:
        # Convert longitude and latitude to float
        longitude = float(document["geocode"]["longitude"])
        latitude = float(document["geocode"]["latitude"])
        
        establishments_collection.update_one(
            {"_id": document["_id"]},
            {"$set": {"geocode.longitude": longitude, "geocode.latitude": latitude}}
        )
        updated_count += 1
    
    remaining_documents = total_documents - updated_count

print("Number of documents updated:", updated_count)


Number of documents updated: 39000


Use `update_many` to convert `RatingValue` to integer numbers.

In [41]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [39]:
# Change the data type from String to Integer for RatingValue
for document in documents_to_update:
    if document["geocode"]["longitude"].replace(".", "", 1).isdigit() and document["geocode"]["latitude"].replace(".", "", 1).isdigit():
        longitude = float(document["geocode"]["longitude"])
        latitude = float(document["geocode"]["latitude"])
        
        establishments_collection.update_one(
            {"_id": document["_id"]},
            {"$set": {"geocode.longitude": longitude, "geocode.latitude": latitude}}
        )
        updated_count += 1
    else:
        print("Skipping document with non-numeric longitude or latitude:", document["_id"])


In [42]:
# Check that the coordinates and rating value are now numbers
document = establishments_collection.find_one({"RatingValue": {"$exists": True}})

if document:
    print("Longitude type:", type(document["geocode"]["longitude"]))
    print("Latitude type:", type(document["geocode"]["latitude"]))
    
    if "RatingValue" in document:
        print("RatingValue type:", type(document["RatingValue"]))
    else:
        print("RatingValue field not found in the document.")
else:
    print("No documents found in the collection.")


Longitude type: <class 'float'>
Latitude type: <class 'float'>
RatingValue type: <class 'str'>
